In [1]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import numpy
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
from tqdm import tqdm
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import Dense, Input,Dropout,GlobalAveragePooling2D
from keras.preprocessing import image_dataset_from_directory
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.models import Sequential


In [3]:
dataset_path = 'D:\Documents\ICS Project II\Datasets\Dermnet'

# Load images 
images = []

In [4]:
#dataset paths
dataset_train_path = os.path.join(dataset_path,'train')
dataset_test_path = os.path.join(dataset_path,'test')

In [5]:
# train and test dataset
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
  dataset_train_path,
  batch_size=128)

dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
  dataset_test_path,
  batch_size=128
)

Found 15557 files belonging to 23 classes.
Found 4002 files belonging to 23 classes.


In [6]:
#labels
dataset_name=dataset_train
class_names=dataset_train.class_names

print(class_names)

['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [7]:
#Image resizing & data standardization
image_size=(246, 246)

dataset_train = dataset_train.map (lambda image, label: (tf.image.resize(image, image_size),label))
dataset_test = dataset_test.map (lambda image, label:(tf.image.resize(image, image_size), label))

In [8]:
#Loading training images

trn_images = []
trn_labels = []
for label in os.listdir(dataset_train_path):
    label_dir = os.path.join(dataset_train_path, label)
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224, 224))
        trn_images.append(image)
        trn_labels.append(label)

#lists->numpy arrays
trn_images = np.array(trn_images)
trn_labels = np.array(trn_labels)

#Loading test images

tst_images = []
tst_labels = []
for label in os.listdir(dataset_test_path):
    label_dir = os.path.join(dataset_test_path, label)
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (224, 224))
        tst_images.append(image)
        tst_labels.append(label)

# Lists -> numpy arrays
tst_images = np.array(tst_images)
tst_labels = np.array(tst_labels)

In [ ]:
#Flattening training images

trn_images_flat = trn_images.reshape(trn_images.shape[0], -1)

#KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(trn_images_flat)

#cluster labels
trn_labels_segmented = kmeans.labels_

In [26]:
# GLCM Feature extraction
def clc_glcm_features(images):
    glcm_features = []

#feature extraction for training & testing

    for image in trn_images:
      glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
      contrast = graycoprops(glcm, 'contrast')[0, 0]
      dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
      homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
      energy = graycoprops(glcm, 'energy')[0, 0]
      correlation = graycoprops(glcm, 'correlation')[0, 0]
      glcm_features.append([contrast, dissimilarity, homogeneity, energy, correlation])

    return np.array(glcm_features)



# Feature vectors->numpy arrays
trn_glcm_features = clc_glcm_features(trn_images)
tst_glcm_features = clc_glcm_features(tst_images)
    

In [52]:
from sklearn.preprocessing import LabelEncoder

#Preprocessing feature vectors
trn_features = trn_glcm_features/np.max(trn_glcm_features)
tst_features = tst_glcm_features/np.max(tst_glcm_features)


reshaped_trn_features = np.reshape(trn_features, (331,47,5))

#Repeat the features to form an image with (224, 224, 3)
trn_images = np.repeat(reshaped_trn_features, 224, axis=1)
trn_images = np.repeat(trn_images, 224, axis=2)
trn_images = np.repeat(trn_images, 3, axis=3)


#inceptionresnetV2
inception_model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224, 3)))

# Global avrg pooling & dense layer for classification
x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

#Identify unique classes
unique_classes = np.unique(trn_images)
num_classes = len(unique_classes)
predictions = Dense(num_classes, activation='softmax')(x)

#Model
model = Model(inputs=inception_model.input, outputs=predictions)

#Freezing layers of inception model
for layer in inception_model.layers:
    layer.trainable = False

#Comp
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

label_encoder = LabelEncoder()
trn_labels_encoded = label_encoder.fit_transform(trn_labels)

#Conversion to one-hot encoding
trn_labels_one_hot = to_categorical(trn_labels_encoded)

#Train
model.fit(reshaped_trn_features, trn_labels_one_hot, batch_size=64, epochs=50, validation_split=0.2, callbacks=[EarlyStopping(patience=3)])

#Test
pred = model.predict(tst_features)
pred_labels = np.argmax(predictions, axis=1)


#Acc and Precision
accuracy = accuracy_score(tst_labels, pred_labels)
precision = precision_score(tst_labels, pred_labels, average='weighted')


#inception_features = inception_model.predict(trn_features)
#inception_features_test = inception_model.predict(tst_features)


MemoryError: Unable to allocate 29.1 GiB for an array with shape (331, 10528, 1120) and data type float64

In [39]:
print(trn_features.size)
print(trn_features.shape)



77785
(15557, 5)


In [ ]:
#CapsNet model
cpsle_model = Sequential()
cpsle_model.add(Dense(64, activation='relu', input_shape=(trn_features.shape[1],)))
cpsle_model.add(Dropout(0.5))
cpsle_model.add(Dense(1, activation='sigmoid'))

#Compilation and training
cpsle_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cpsle_model.fit(inception_features, trn_labels, epochs = 100 , batch_size =32)

#Model evaluation
inception_predictions = cpsle_model.predict_classes(inception_features_test)
accuracy = accuracy_score(tst_labels, inception_predictions)
precision = precision_score(tst_labels, inception_predictions)

In [ ]:
print("Precision: {:.2f}%".format(precision * 100))
print("Accuracy: {:.2f}%".format(accuracy * 100))